In [1]:
!pip install opencc 
import requests
from bs4 import BeautifulSoup
import pandas as pd
from opencc import OpenCC
import random
import time

class NYtime_crawl:
	def __init__(self, x):
		self.page = x  #初始欲下載的頁數 (p.s 紐時1頁有20篇新聞)
		self.titles = []
		self.sub_title = []
		self.date = []
		self.contents = []
		self.urls = []
		self.count = 1
		self.url = 'https://cn.nytimes.com/world/'
		
	def crawl(self):
		my_headers = [
					"Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36",
					"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36",
					"Mozilla/5.0 (Windows NT 6.1; WOW64; rv:30.0) Gecko/20100101 Firefox/30.0",
					"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/537.75.14",
					"Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; Win64; x64; Trident/6.0)",
					'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11',
					'Opera/9.25 (Windows NT 5.1; U; en)',
					'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)',
					'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.5 (like Gecko) (Kubuntu)',
					'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.8.0.12) Gecko/20070731 Ubuntu/dapper-security Firefox/1.5.0.12',
					'Lynx/2.8.5rel.1 libwww-FM/2.14 SSL-MM/1.4.1 GNUTLS/1.2.9',
					"Mozilla/5.0 (X11; Linux i686) AppleWebKit/535.7 (KHTML, like Gecko) Ubuntu/11.04 Chromium/16.0.912.77 Chrome/16.0.912.77 Safari/535.7",
					"Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:10.0) Gecko/20100101 Firefox/10.0 "
					]
		r = requests.get(self.url, headers = {'User-Agent':random.choice(my_headers)})
		html = BeautifulSoup(r.text, 'html.parser') 
		# simplified chinese to tranditional chinese
		cc = OpenCC('s2tw')

		# Get next_page_link
		nextpage = html.find('li', class_="next").a['href']

		# Get title
		news = html.find_all('h3')
		for new in news:
		  title = cc.convert(new.a.text)
		  self.titles.append(title)

		# Get sub-title
		news = html.find_all('p', class_="summary")
		for new in news:
		  sub = cc.convert(new.text.replace('阅读更多', ''))
		  self.sub_title.append(sub)

		# Get link
		news = html.find_all('h3')
		for new in news:
			link = 'https://cn.nytimes.com' + new.a['href']
			self.urls.append(link)
			
			# request for each news & analyse
			r = requests.get(link, headers = {'User-Agent':random.choice(my_headers)})
			html = BeautifulSoup(r.text, 'html.parser')

			# filter for non-news
			try:
				# Get time
				self.date.append(html.find('time').text)
				# Get content
				sentences = html.find_all('div', class_="article-paragraph")
				content = ''.join([sentence.text for sentence in sentences][1:])
				self.contents.append(cc.convert(content))
			except:
				self.date.append('2020年5月15日')
				sentences = html.find_all('div', class_="article-paragraph")
				content = ''.join([sentence.text for sentence in sentences][1:])
				self.contents.append(cc.convert(content))
			print('第' + str(len(self.urls)) + '篇新聞下載完成')
		return nextpage

	def df_to_csv(self):
		fordf = [self.titles, self.sub_title, self.date, self.contents, self.urls]
		c = ['title', 'sub_title', 'date', 'content', 'url']	
		return pd.DataFrame(fordf, index=c).T.to_csv('news.csv', encoding = 'utf-8', index = False)

	def get_data(self):
		while self.count <= self.page:
			print('======第' + str(self.count) + '頁======')
			try:
				self.url = 'https://cn.nytimes.com' + self.crawl()
			except:
				print('=====顆顆抓太多筆顯示403禁止囉=====')
				print('休息5分鐘，再從第' + str(self.count) + '頁開始吧')
				time.sleep(300)
				self.url = 'https://cn.nytimes.com/world/' + str(self.count)
				self.url = 'https://cn.nytimes.com' + self.crawl()	
			self.df_to_csv()
			self.count += 1
		
			
# 操作實例
X = NYtime_crawl(50) #初始設定爬50頁(共1000篇新聞)
X.get_data()

======第1頁======
第1篇新聞下載完成
第2篇新聞下載完成
第3篇新聞下載完成
第4篇新聞下載完成
第5篇新聞下載完成
第6篇新聞下載完成
第7篇新聞下載完成
第8篇新聞下載完成
第9篇新聞下載完成
第10篇新聞下載完成
第11篇新聞下載完成
第12篇新聞下載完成
第13篇新聞下載完成
第14篇新聞下載完成
第15篇新聞下載完成
第16篇新聞下載完成
第17篇新聞下載完成
第18篇新聞下載完成
第19篇新聞下載完成
第20篇新聞下載完成
======第2頁======
第21篇新聞下載完成
第22篇新聞下載完成
第23篇新聞下載完成
第24篇新聞下載完成
第25篇新聞下載完成
第26篇新聞下載完成
第27篇新聞下載完成
第28篇新聞下載完成
第29篇新聞下載完成
第30篇新聞下載完成
第31篇新聞下載完成
第32篇新聞下載完成
第33篇新聞下載完成
第34篇新聞下載完成
第35篇新聞下載完成
第36篇新聞下載完成
第37篇新聞下載完成
第38篇新聞下載完成
第39篇新聞下載完成
第40篇新聞下載完成
======第3頁======
第41篇新聞下載完成
第42篇新聞下載完成
第43篇新聞下載完成
第44篇新聞下載完成
第45篇新聞下載完成
第46篇新聞下載完成
第47篇新聞下載完成
第48篇新聞下載完成
第49篇新聞下載完成
第50篇新聞下載完成
第51篇新聞下載完成
第52篇新聞下載完成
第53篇新聞下載完成
第54篇新聞下載完成
第55篇新聞下載完成
第56篇新聞下載完成
第57篇新聞下載完成
第58篇新聞下載完成
第59篇新聞下載完成
第60篇新聞下載完成
======第4頁======
第61篇新聞下載完成
第62篇新聞下載完成
第63篇新聞下載完成
第64篇新聞下載完成
第65篇新聞下載完成
第66篇新聞下載完成
第67篇新聞下載完成
第68篇新聞下載完成
第69篇新聞下載完成
第70篇新聞下載完成
第71篇新聞下載完成
第72篇新聞下載完成
第73篇新聞下載完成
第74篇新聞下載完成
第75篇新聞下載完成
第76篇新聞下載完成
第77篇新聞下載完成
第78篇新聞下載完成
第79篇新聞下載完成
第80篇新聞下載完成
======第5頁======
第81篇新聞下載完成
第82篇新聞下載完成
第83篇新聞下載完成
第84篇新聞下載完成
第85篇新

In [0]:
class drawable_summary():
    def __init__(self, dataframe, rank_algorithm, sum_length, num_of_keyword, remove_word):
        self.dataframe = dataframe
        self.rank_algorithm = rank_algorithm
        self.sum_length = sum_length
        self.num_of_keyword = num_of_keyword
        self.remove_word = [remove_word]

        
    def choose_algorithm(self, content): # 選擇演算法
        if self.rank_algorithm == 'tf-idf':
            keyword_tf = jieba.analyse.extract_tags(content,
                                                    topK=self.num_of_keyword,
                                                    withWeight=True,
                                                    allowPOS=())
            self.keyword = keyword_tf
            
        elif self.rank_algorithm == 'text_rank':
            keyword_Tr = jieba.analyse.textrank(content,
                                                topK=self.num_of_keyword,
                                                withWeight=True,
                                                allowPOS=('ns', 'n', 'vn', 'v'))
            self.keyword = keyword_Tr
            
        elif self.rank_algorithm == 'mix':
            keyword_tf = jieba.analyse.extract_tags(content,
                                                    topK=self.num_of_keyword,
                                                    withWeight=True,
                                                    allowPOS=())
            keyword_Tr = jieba.analyse.textrank(content,
                                                topK=self.num_of_keyword,
                                                withWeight=True,
                                                allowPOS=('ns', 'n', 'vn', 'v'))
            mix_key = {x[0]:x[1] for x in keyword_tf} 
            for word in keyword_Tr:
                if word[0] in mix_key.keys():
                    mix_key[word[0]] += word[1]
                else:
                    mix_key[word[0]] = word[1]
        
            mix_key = [(x,mix_key[x]) for x in mix_key]
            self.keyword = mix_key
            
    def give_score(self, word_after_jieba): # 給每個句子分數
        def none_empty(x):  # 避免出現空直的狀況
            return x.strip()
        process_data = [sent.split(' ')for sent in word_after_jieba.split('。')]
        process_data = [list(filter(none_empty, x)) for x in process_data if len(list(filter(none_empty, x))) > 0]
        
        total_info = []
        for sent in process_data:
            score = 0
            for word in self.keyword:
                if (word[0] in sent) & (word[0] not in self.remove_word):   # 考慮停用詞
                    score += word[1]
            total_info.append(tuple([''.join(sent),score]))
            
        self.sent_score = total_info
        
    def draw_length(self):   # 處理抽取長度
        score_list = sorted([x[1] for x in self.sent_score], reverse = True)   # 分數做列表 
        average_score = sum(score_list)/len(score_list)                        # 平均分數
        percentile = np.percentile(score_list, [25, 50, 75])                   #  各四分位距
        
        if self.sum_length.isdigit():
            threshold = score_list[int(self.sum_length)-1]
        elif self.sum_length == 'avg':
            threshold = average_score
        elif self.sum_length == 'Q1':
            threshold = percentile[0]
        elif self.sum_length == 'Q2':
            threshold = percentile[1]
        elif self.sum_length == 'Q3':
            threshold = percentile[2]
            
        summary = [x[0] for x in self.sent_score if x[1] >= threshold]
        summary = '。'.join(summary)
        return summary
    
    def make_summary(self):
        self.dataframe['summary1'] = None
        for i in range(len(self.dataframe)):
            self.choose_algorithm(self.dataframe.loc[i, 'content'])
            self.give_score(self.dataframe.loc[i, 'jieba'])
            self.dataframe.loc[i,'summary1'] = self.draw_length()
        return self.dataframe

# summary = drawable_summary(df,'text_rank', 'Q3',20,'交貨').make_summary() # 1068、  Q3 = 269, Q2 = 367, Q1 = 525
# print(len(summary['jieba'][0]), len(summary['summary1'][0]))
# print(summary['summary1'][0])
# summary